In [ ]:
import numpy as np
import pandas as pd
import joblib
from google.colab import drive
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split, cross_val_score
import xgboost as xgb
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
drive.mount('/content/drive')

Mounted at /content/drive


1. Demonstrate the data preparation & feature extraction process [5]
2. Create feature subsets that show maximum correlation with the dependent variable. [5]

In [ ]:
players_21 = pd.read_csv('/content/drive/My Drive/Classification/players_21.csv')
players_21

In [ ]:
players_22 = pd.read_csv('/content/drive/My Drive/Classification/players_22.csv')
players_22

In [ ]:
players_21.shape

In [ ]:
players_21.describe()

In [ ]:
players_21.info()

In [ ]:
players_21.columns

In [ ]:
players_21.head()

Data cleaning

In [ ]:
#dropping columns with more than 30% missing values
missing_percentage = players_21.isnull().sum() / len(players_21) * 100
columns_to_drop = missing_percentage[missing_percentage > 30].index
players_21.drop(columns=columns_to_drop, inplace=True)

In [ ]:
#dropping URLs and IDs since they do not contribute to player rating
url_columns = [col for col in players_21.columns if 'url' in col.lower() or 'id' in col.lower()]
players_21.drop(columns=url_columns, inplace=True)
players_21.info()

In [ ]:
# Splitting into numeric and categorical
numeric_data = players_21.select_dtypes(include=['int64', 'float64'])
categorical_data = players_21.select_dtypes(include=['object'])

In [ ]:
# Imputing numeric data with the mean
mean_imputer = SimpleImputer(strategy='mean')
numeric_data_imputed = pd.DataFrame(mean_imputer.fit_transform(numeric_data), columns=numeric_data.columns)

In [ ]:
# Imputing categorical data with the mode
mode_imputer = SimpleImputer(strategy='most_frequent')
categorical_data_imputed = pd.DataFrame(mode_imputer.fit_transform(categorical_data), columns=categorical_data.columns)

In [ ]:
from sklearn.preprocessing import LabelEncoder
# Label encoding the imputed categorical data using Pandas
categorical_data_imputed = categorical_data_imputed.apply(lambda col: col.astype('category').cat.codes)

In [ ]:
# Combining the imputed data back together
players_21_imputed = pd.concat([numeric_data_imputed, categorical_data_imputed], axis=1)
players_21_imputed = players_21_imputed[players_21.columns]
players_21_imputed

In [ ]:
#doing correlation to choose highly related columns
correlation_matrix = players_21_imputed.corr()
overall_correlation = correlation_matrix['overall'].sort_values(ascending=False)
overall_correlation

In [ ]:
# Extract columns with correlation greater than 0.5
highly_correlated_features = [feature for feature, corr_value in overall_correlation.items() if 0.5 < corr_value < 1.0]
highly_correlated_features

In [ ]:
# Dropping these columns since they all fall under player positions and do not directly influence player rating
columns_to_drop = ['lcm', 'cm', 'rcm', 'lm', 'rm', 'cam', 'ram', 'lam', 'lf', 'rf', 'cf',
                   'rw', 'lw', 'st', 'ls', 'rs', 'rwb', 'lwb', 'ldm', 'cdm', 'rdm', 'rb', 'lb', 'cb','lcb','rcb',]
highly_correlated_features = [feature for feature in highly_correlated_features if feature not in columns_to_drop]
highly_correlated_features

In [ ]:
#final selected columns after correlation
selected_columns= list(highly_correlated_features)
selected_columns

In [62]:
#Selected columns from correlation with 'age','shooting','skill_ball_control','work_rate' combined as they influence player rating
selected_columns= ['movement_reactions','mentality_composure','passing','potential','release_clause_eur',
                   'dribbling','wage_eur','power_shot_power','value_eur','mentality_vision',
                   'attacking_short_passing','age','shooting','skill_ball_control','work_rate']

In [ ]:
#Scaling
# Extracting the selected features
selected_features = players_21_imputed[selected_columns]
scaler = StandardScaler()
scaled_features = scaler.fit_transform(selected_features)
# Converting the scaled features back to a DataFrame for better visualization and further operations
scaled_features_df = pd.DataFrame(scaled_features, columns=selected_columns)
scaled_features_df.head()

In [ ]:
joblib.dump(scaler, 'scaler_model.joblib') # to be used when testing

['scaler_model.joblib']

3. Create and train a suitable machine learning model with cross-validation that can predict a player's rating. [5]

4. Measure the model's performance and fine-tune it as a process of optimization. [5]

In [ ]:
X = scaled_features_df
y = players_21_imputed['overall']

In [ ]:
#split into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
#Creating and Training the random forest Model using Cross-Validation
rf = RandomForestRegressor(n_estimators=100, random_state=42)
# Use cross-validation to train
cross_val_scores = cross_val_score(rf, X_train, y_train, cv=5, scoring='neg_mean_absolute_error')
print(f"Average MAE from Cross Validation: {-cross_val_scores.mean()}")

Average MAE from Cross Validation: 0.28152424942263293


In [ ]:
#Creating and Training the gradient boost Model using Cross-Validation
gboost_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
# Train the model with cross-validation
scores = cross_val_score(gboost_model, X, y, scoring="neg_mean_absolute_error", cv=5)
average_mae_gboost = -scores.mean()
print(f"Average MAE from Cross Validation with Gradient Boosting: {average_mae_gboost}")

Average MAE from Cross Validation with Gradient Boosting: 2.3178540533633143


In [ ]:
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100)
# Train the model with cross-validation
xgb_scores = cross_val_score(xgb_model, X, y, cv=5, scoring='neg_mean_absolute_error')
xgb_mean_mae = -xgb_scores.mean()
print(f"XGBoost Cross-validated Mean Absolute Error: {xgb_mean_mae}")

XGBoost Cross-validated Mean Absolute Error: 2.149413922244201


In [ ]:
#fine tuning random forest
param_grid_rf = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [None, 3, 5],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2, 4]
}
rf_model = RandomForestRegressor(random_state=42)
grid_search_rf = GridSearchCV(rf_model, param_grid_rf, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
grid_search_rf.fit(X_train, y_train)
print("Best Parameters for Random Forest:")
print(grid_search_rf.best_params_)
print(f"Best MAE for Random Forest: {-grid_search_rf.best_score_}")

Best Parameters for Random Forest:
{'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Best MAE for Random Forest: 0.27939194985153415


In [ ]:
# Predicting using the optimized Random Forest model
rf_optimized_predictions = grid_search_rf.best_estimator_.predict(X_test)
# Calculate MAE for optimized Random Forest model
rf_optimized_mae = mean_absolute_error(y_test, rf_optimized_predictions)
print(f"Optimized Random Forest MAE: {rf_optimized_mae}")

Optimized Random Forest MAE: 0.2520229612034837


In [ ]:
#fine tuning Gradient boost
param_grid_gb = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.05],
    'max_depth': [4, 5],
    'min_samples_split': [5, 10],
    'min_samples_leaf': [2, 4],
    'subsample': [0.8, 0.9]
}

gb_model = GradientBoostingRegressor()

grid_search_gb = GridSearchCV(gb_model, param_grid_gb, cv=5,
                              scoring='neg_mean_absolute_error', n_jobs=-1)
grid_search_gb.fit(X_train, y_train)
print("Best Parameters for Gradient Boosting:")
print(grid_search_gb.best_params_)
print(f"Best MAE for Gradient Boosting: {-grid_search_gb.best_score_}")

Best Parameters for Gradient Boosting:
{'learning_rate': 0.05, 'max_depth': 5, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 200, 'subsample': 0.9}
Best MAE for Gradient Boosting: 0.3964194121737254


In [ ]:
# Predicting using the optimized Gradient Boosting model
gb_optimized_predictions = grid_search_gb.best_estimator_.predict(X_test)

# Calculate MAE for optimized Gradient Boosting model
gb_optimized_mae = mean_absolute_error(y_test, gb_optimized_predictions)

print(f"Optimized Gradient Boosting MAE: {gb_optimized_mae}")

Optimized Gradient Boosting MAE: 0.38231359002931514


In [ ]:
#fine tuning XG boost
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.05, 0.1]
}

grid_search = GridSearchCV(xgb_model, param_grid, cv=5, scoring='neg_mean_absolute_error')
grid_search.fit(X_train, y_train)

print(f"Best parameters for XGBoost: {grid_search.best_params_}")
print(f"Best MAE for XGBoost: {-grid_search.best_score_}")


Best parameters for XGBoost: {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 150}
Best MAE for XGBoost: 0.31196077146061335


In [ ]:
best_xgb_model = grid_search.best_estimator_

best_xgb_model.fit(X_train, y_train)
xgb_test_predictions = best_xgb_model.predict(X_test)
xgb_test_mae = mean_absolute_error(y_test, xgb_test_predictions)

print(f"Test MAE after fine-tuning for XGBoost: {xgb_test_mae}")

Test MAE after fine-tuning for XGBoost: 0.29266076529677293


combining all 3 models

In [ ]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression

# Base models
base_models = [
    ('rf', grid_search_rf.best_estimator_),
    ('gboost', grid_search_gb.best_estimator_),
    ('xgb', best_xgb_model)
]
meta_model = LinearRegression()
stacked_model = StackingRegressor(estimators=base_models, final_estimator=meta_model)
stacked_model.fit(X_train, y_train)

# Making predictions and computing MAE:
stacked_predictions = stacked_model.predict(X_test)
stacked_mae = mean_absolute_error(y_test, stacked_predictions)

print(f"Stacked Ensemble MAE: {stacked_mae}")


Stacked Ensemble MAE: 0.262490318965769


5.Use the data from another season(players_22) which was not used during the training to test how good is the model. [5]

In [ ]:
players_22.head()

In [ ]:
players_22.describe()

In [ ]:
players_22.info()

In [ ]:
#dropping columns with more than 30% missing values
missing_percentage = players_22.isnull().sum() / len(players_22) * 100
columns_to_drop = missing_percentage[missing_percentage > 30].index
players_22.drop(columns=columns_to_drop, inplace=True)
#dropping URLs and IDs since they do not contribute to player rating
url_columns = [col for col in players_22.columns if 'url' in col.lower() or 'id' in col.lower()]
players_22.drop(columns=url_columns, inplace=True)
players_22.info()

In [ ]:
# Splitting into numeric and categorical
numerical_data = players_22.select_dtypes(include=['int64', 'float64'])
categoric_data = players_22.select_dtypes(include=['object'])
# Imputing numeric data with the mean
mean_imputer = SimpleImputer(strategy='mean')
numerical_data_imputed = pd.DataFrame(mean_imputer.fit_transform(numerical_data), columns=numerical_data.columns)
# Imputing categorical data with the mode
mode_imputer = SimpleImputer(strategy='most_frequent')
categoric_data_imputed = pd.DataFrame(mode_imputer.fit_transform(categoric_data), columns=categoric_data.columns)

In [ ]:
from sklearn.preprocessing import LabelEncoder
# Label encoding the imputed categorical data using Pandas
categoric_data_imputed_encoded = categoric_data_imputed.apply(lambda col: col.astype('category').cat.codes)

In [ ]:
# Combining the imputed data back together
players_22_imputed = pd.concat([numerical_data_imputed, categoric_data_imputed_encoded], axis=1)
players_22_imputed = players_22_imputed[players_22.columns]
players_22_imputed

In [ ]:
chosen_columns = players_22[highly_correlated_features]
chosen_columns

In [ ]:
chosen_columns_22= ['movement_reactions','mentality_composure','passing','potential','release_clause_eur',
                   'dribbling','wage_eur','power_shot_power','value_eur','mentality_vision',
                   'attacking_short_passing','age','shooting','skill_ball_control','work_rate']# Same as 21

In [ ]:
#Scaling
# Extracting the selected features
selected_features_22 = players_22_imputed[chosen_columns_22]
scaler = joblib.load('scaler_model.joblib','r')
scaled_features = scaler.transform(selected_features_22)
# Converting the scaled features back to a DataFrame for better visualization and further operations
scaled_features_22_df = pd.DataFrame(scaled_features, columns=chosen_columns_22)
scaled_features_22_df.head()

In [ ]:
actual_ratings_22 = players_22['overall']


In [63]:
# 2. Evaluation using Mean Absolute Error
predicted_ratings_22 = np.floor(stacked_model.predict(scaled_features_22_df))
mae_22 = mean_absolute_error(actual_ratings_22, predicted_ratings_22)
print(f"Mean Absolute Error on players_22 data using stacked ensemble model is : {mae_22}")

Mean Absolute Error on players_22 data using stacked ensemble model is : 0.5861531264618743


In [64]:
predicted_ratings_22 = np.floor(grid_search_rf.best_estimator_.predict(scaled_features_22_df))
mae_22 = mean_absolute_error(actual_ratings_22, predicted_ratings_22)
print(f"Mean Absolute Error on players_22 data using trained random forest: {mae_22}")

Mean Absolute Error on players_22 data using trained random forest: 0.6103227818493685


In [65]:
predicted_ratings_22 = np.floor(grid_search_gb.best_estimator_.predict(scaled_features_22_df))
mae_22 = mean_absolute_error(actual_ratings_22, predicted_ratings_22)
print(f"Mean Absolute Error on players_22 data using gradient boost: {mae_22}")

Mean Absolute Error on players_22 data using gradient boost: 0.6050210509901762


In [66]:
predicted_ratings_22 = np.floor(best_xgb_model.predict(scaled_features_22_df))
mae_22 = mean_absolute_error(actual_ratings_22, predicted_ratings_22)
print(f"Mean Absolute Error on players_22 data using XG boost: {mae_22}")

Mean Absolute Error on players_22 data using XG boost: 0.5804875513280316


Best model, with lowest MAE after testing is the XGBoost,although value very close to stacked ensemble model. Hence efficient

6. Deployment

In [ ]:
!pip install joblib
from joblib import dump

In [ ]:
dump(stacked_model, 'trained_ensemble_model.joblib')

['trained_ensemble_model.joblib']

In [ ]:
dump(best_xgb_model, 'best_model.joblib')

['best_model.joblib']

In [ ]:
import sklearn
sklearn.__version__

'1.2.2'